In [1]:
import sys
sys.path.insert(0, "..")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IA.utils import mapping, parameter_range
from scipy import stats
import math

In [2]:
!ls -lah analysis/not_uploaded/IA/ | grep regress

-rw-r--r-- 1 janpf janpf 902M Oct 27 01:04 .scratch.ckpts.IA.pexels.scores-None.change_regress.epoch-8.pth.txt
-rw-r--r-- 1 janpf janpf 936M Oct 27 01:05 .scratch.ckpts.IA.pexels.scores-one.change_regress.epoch-5.pth.txt
-rw-r--r-- 1 janpf janpf 935M Oct 27 01:05 .scratch.ckpts.IA.pexels.scores-one.change_regress.epoch-9.pth.txt


In [5]:
df = pd.read_csv("analysis/not_uploaded/IA/.scratch.ckpts.IA.pexels.scores-one.change_regress.epoch-9.pth.txt")

for col in df.columns:
    if not "score" in col:
        continue

    df[col] = df[col].apply(lambda row: eval(row)[0])

print(df.columns)
df

Index(['img', 'distortion', 'level', 'score', 'styles_contrast_strength',
       'styles_brightness_strength', 'styles_saturation_strength',
       'styles_shadows_strength', 'styles_highlights_strength',
       'styles_exposure_strength', 'styles_vibrance_strength',
       'styles_temperature_strength', 'styles_tint_strength',
       'technical_jpeg_compression_strength',
       'technical_defocus_blur_strength', 'technical_motion_blur_strength',
       'technical_pixelate_strength', 'technical_gaussian_noise_strength',
       'technical_impulse_noise_strength', 'composition_rotate_strength',
       'composition_hcrop_strength', 'composition_vcrop_strength',
       'composition_leftcornerscrop_strength',
       'composition_rightcornerscrop_strength', 'composition_ratio_strength'],
      dtype='object')


,img,distortion,level,score,styles_contrast_strength,styles_brightness_strength,styles_saturation_strength,styles_shadows_strength,styles_highlights_strength,styles_exposure_strength,...,technical_motion_blur_strength,technical_pixelate_strength,technical_gaussian_noise_strength,technical_impulse_noise_strength,composition_rotate_strength,composition_hcrop_strength,composition_vcrop_strength,composition_leftcornerscrop_strength,composition_rightcornerscrop_strength,composition_ratio_strength
0,5/3/c/abbey-glass-religion-architecture-106158...,original,0.0,0.461914,0.245361,-0.075195,-0.041626,-0.066406,0.291748,-0.002167,...,0.298584,0.582031,0.311035,0.277832,-0.057190,-0.269287,0.160889,0.153809,0.167480,0.028122
1,9/4/4/abraham-lincoln-lincoln-memorial-washing...,original,0.0,0.756348,0.207764,-0.211304,0.346680,-0.062012,-0.141113,-0.161377,...,-0.068359,0.339844,0.010010,0.010254,0.219849,0.003036,-0.238159,-0.232910,-0.227783,0.070862
2,a/5/9/abstract-background-backgrounds-botany-8...,original,0.0,0.890137,0.606934,-0.140747,0.610352,-0.027466,0.158447,0.600098,...,-0.284424,-0.229004,-0.626953,-0.625000,0.163330,-0.415283,0.175049,0.122131,0.247559,0.316406
3,2/2/c/adler-bird-bird-of-prey-raptor-53587.jpeg,original,0.0,0.512207,0.916992,-0.916016,0.281738,-0.796875,0.280762,-0.354980,...,-0.104492,-0.019531,0.066406,-0.012695,-0.031586,-0.084473,-0.413818,-0.402344,-0.459961,0.049896
4,a/0/7/adult-cute-face-female-41522.jpeg,original,0.0,0.750977,0.173828,0.111206,0.050812,-0.588379,1.097656,0.172729,...,-0.112549,0.289062,-0.173584,-0.132324,0.093811,-0.058533,0.176270,0.122375,0.218262,0.088989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609995,0/d/b/wood-painting-window-cactus-6689.jpg,ratio,5.0,0.513672,0.213379,-0.258545,0.170532,-0.634766,0.247559,-0.270508,...,0.223633,0.343262,0.041504,0.030029,0.329590,-0.066895,-0.510254,-0.518066,-0.507324,-0.042450
2609996,0/1/0/wood-sky-forest-autumn.jpg,ratio,5.0,0.326904,0.683105,-0.647949,0.239746,-0.371582,0.209229,-0.047180,...,0.095947,0.177246,0.201416,0.142578,-0.059601,0.329346,-0.503906,-0.480713,-0.495117,0.131958
2609997,0/d/2/wood-stool-wooden-wine.jpg,ratio,5.0,0.483154,0.068420,-0.022736,0.551270,-0.170166,0.175293,0.088257,...,0.135010,0.461182,0.099487,0.082397,0.053741,-0.215576,-0.032806,-0.012268,-0.031128,0.572266
2609998,b/5/e/wood-tree-brown-bark.jpg,ratio,5.0,0.240967,0.557129,-0.431641,0.061462,-0.634277,0.416016,0.076660,...,0.197021,0.388672,0.205444,0.132812,-0.008591,0.314941,0.023285,-0.005432,0.042664,-0.265137


In [26]:
distortion = "brightness"
polarity = "pos"

orig_best = 0
overall = 0


original_df = df[df["distortion"] == "original"]
if distortion in parameter_range:
    original_df["level"] = parameter_range[distortion]["default"]

parameter_df = df[df["distortion"] == distortion]
corr_df = pd.concat([parameter_df, original_df])[["img", "distortion", "level", "score"]]

if distortion in parameter_range:
    default = parameter_range[distortion]["default"]
else:
    default = 0

if polarity == "pos":
    corr_df = corr_df[corr_df["level"] >= default]
else:
    corr_df = corr_df[corr_df["level"] <= default]

corr_df["level"] = corr_df["level"].apply(lambda x: abs((x) - (default)))

for i, f in enumerate(df["img"].unique()):
    corr_df_img = corr_df[corr_df["img"] == f]
    
    max_val = float(max(corr_df_img["score"]))
    ori_val = float(corr_df_img[corr_df_img["distortion"] == "original"].iloc[0]["score"])
    
    overall +=1
    if math.isclose(max_val,ori_val):
        orig_best +=1

orig_best, overall

(6802, 14999)

In [27]:
orig_best / overall

0.45349689979331953

In [28]:
overall/6

2499.8333333333335

In [29]:
1/6

0.16666666666666666